<center><h1>Where is Elon?!</h1></center>

Elon musk's plane tracker, will include:
- Data scraping
- Data visualization

Jet information
- Registration: N628TS
- ICAO hex number: A835AF
- Category: A3 Large (75000 to 300000 lb)
- Bought in 2006

Source is ADS-B exchange (All proper APIs are priced)
- https://globe.adsbexchange.com
- https://globe.adsbexchange.com/?icao=a835af&lat=37.731&lon=-122.215&zoom=1.0&showTrace=2023-02-08 Using their replay function for day -> showTrace=YYYY-MM-DD
- https://globe.adsbexchange.com/?replay=2023-02-16-11:46&lat=40.560&lon=-73.660&zoom=9.0 (Deprecated) Using their replay function -> replay=YYYY-MM-DD-HH:MM

- Beginning of data seems to be on 2020-01-19

Settings:
- replay=YYYY-MM-DD-HH:MM
- icao=a835af
- lat=0
- lon=0
- zoom=2.0

Javascript variable:
- selectedPlanes
    - fullTrace

<!-- 
other info (temporary)
ads-b exchange replay function
https://globe.adsbexchange.com/?replay=2023-02-16-11:46&lat=40.560&lon=-73.660&zoom=9.0


maybe:
https://globe.adsbexchange.com/?replay=2023-03-01&icao=a835af


https://globe.adsbexchange.com/?icao=a835af&lat=37.731&lon=-122.215&zoom=1.0&showTrace=2023-02-08 <- we got a winner!!!!!

console:
selectedPlanes()
-> fullTrace
-> trace -->

<h2>Imports and settings</h2>

In [32]:
from selenium import webdriver

import json

from time import sleep
import sys

from datetime import datetime, timedelta
from math import floor

In [7]:
#Scraper settings
targetIcao = "a835af"
startDate = datetime.strptime("2020/01/19", '%Y/%m/%d').date()
endDate = datetime.now().date() # datetime.strptime("2023/02/08", '%Y/%m/%d').date()

lat = "0.000"
lon = "0.000"
zoom = "2.0"

#Saving settings
save_file = "tracking_data.json"

In [8]:
#Other data
javaScript_start_time = datetime.strptime("1970/01/01", '%Y/%m/%d') #Time delta in seconds

<h2>Scrap data</h2>

Data from - Selected Planes() -> showTrace

Problem -> Circular reference in javascript object so can't return it </n>
Solution -> https://stackoverflow.com/a/10455320/454773 get rid of circular reference + parse to json

Trace data as:
- list of values + dict
    - [<b>time</b> (javscript time), <b>latitude</b>, <b>longitude</b>, <b>altitude/'ground'</b>, <b>ground speed (kt)</b>, ground track, ?, ?, <b>dict</b>, transponder type (I think), geometric altitude, ?, indicated speed, ?]
    - dict -> keys depend on ground or air + sometimes no info
        - Gonna take: 'flight' (flight number), 'alt_geom' (geometric altitude), 'ias' (indicated speed), 'tas' (true speed), 'mach', 'wd' (wind direction), 'ws' (wind speed), 'track' (ground track), 'mag_heading' (magnetic heading), 'true_heading', 'baro_rate' (barometric climb (ft/min)), 'geom_rate' (rate of climb or decent ft/min)

ground track -> aircraft direction above ground (?)
geometric altitude -> The geometric GPS derived altitude referenced to WGS84 ellipsoid <!-- (or reference EGM96 when enabled in the settings) -->

Data Storage -> Saved in dictionary
- key -> date time information
- value -> dict {lat, lon, baro_alt, grd_speed, extra : dict2}
    - dict2 -> dict {flight_num, geom_alt, geom_rate, baro_rate, indicated_spd, true_spd, mach, wind_dir, wind_spd, grd_track, mag_heading, true_heading} (Not always complete)

In [9]:
#Utility functions
util_bld_dict2_ins = ["flight", "alt_geom", "geom_rate", "baro_rate", "ias", "tas", "mach", "wd", "ws", "track", "mag_heading", "true_heading"]
util_bld_dict2_outs = ["flight_num", "geom_alt", "geom_rate", "baro_rate", "indicated_spd", "true_spd", "mach", "wind_dir", "wind_spd", "grd_track", "mag_heading", "true_heading"]

def build_dict(lin):
    info = {}
    info["lat"] = lin[1] if len(lin) > 1 else None
    info["lon"] = lin[2] if len(lin) > 2 else None
    info["baro_alt"] = lin[3] if len(lin) > 3 else None
    info["grd_speed"] = lin[4] if len(lin) > 4 else None
    if len(lin) < 9 or lin[8] == None:
        info["extra"] = None
    else:
        extra = {}
        for i in range(len(util_bld_dict2_ins)):
            v = lin[8].get(util_bld_dict2_ins[i])
            if v != None:
                extra[util_bld_dict2_outs[i]] = v
        info["extra"] = extra
    return info

def get_java_time(dt):
    return datetime.strptime("1970/01/01", '%Y/%m/%d') + timedelta(seconds=floor(dt))

def has_trace(obj):
    return len(obj) > 0 and obj[0].get("fullTrace") != None and obj[0]["fullTrace"].get("trace") != None

In [ ]:
import sys

def progressbar(p, prefix="", size=60, out=sys.stdout): # Python3.3+
    x = int(size*p)
    print("{}[{}{}] {}%".format(prefix, "#"*x, "."*(size-x), int(p*100)), end='\r', file=out, flush=True)

In [10]:
driver = webdriver.Chrome()

jss = '''const getCircularReplacer = () => {
  const seen = new WeakSet();
  return (key, value) => {
    if (typeof value === "object" && value !== null) {
      if (seen.has(value)) {
        return;
      }
      seen.add(value);
    }
    return value;
  };
};

return JSON.stringify(selectedPlanes(), getCircularReplacer());
'''

#Declare data storage
data = {}

delta = endDate - startDate
for i in range(delta.days + 1):
    #Get webpage
    driver.get("https://globe.adsbexchange.com/?icao={}&lat={}&lon={}&zoom={}&showTrace={}".format(targetIcao, lat, lon, zoom, startDate + timedelta(days=i)))

    #Give enough time for javascript execution + information return
    c = 0
    while (c < 5):
      sleep(1)
      js_data = json.loads(driver.execute_script(jss))
      c += 10 if has_trace(js_data) else 1

    if(has_trace(js_data)):

        for l in js_data[0]['fullTrace']['trace']:
            t = get_java_time(l[0])
            data[str(t)] = build_dict(l)
    
driver.close()

<h2>Save data</h2>

Data saved in json file

In [ ]:
json_data = json.dumps(data, indent=4)
 
# Writing to sample.json
with open(targetIcao + "_" + save_file, "w") as target_file:
    target_file.write(json_data)

<h2>Load data</h2>

In [1]:
#Parameters
load_location = "./a835af_tracking_data_2020-01-19_2023-02-26.json"

In [4]:
with open(load_location, 'r') as f:
    json_data = json.load(f)

<h2>Display data</h2>

1. Display succeding locations where plane stopped

In [5]:
#Dependencies
from geopy.geocoders import Nominatim

In [37]:
#1. Get locations of plane
destinations_list = []

# Initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

# Initialize progress bar
n = len(json_data)
c = 0

toolbar_width = 40

#Run through data
last_loc = None
for date in json_data:
    entry = json_data[date]

    if(entry["baro_alt"] == "ground"):
        location = geolocator.reverse( (entry["lat"] , entry["lon"]) )

        if(location != None):
            location = location.raw['address']
            city = location.get('city', '')
            state = location.get('state', '')
            country = location.get('country', '')
            
            if( len(city) > 0 and len(state) > 0 and len(country) > 0):
                location_str = "{}, {} {}".format(location["city"], location["state"], location["country"])

                if location_str != last_loc:
                    destinations_list.append([date, location_str])
                    last_loc = location_str
    
    if ((c * 100) % n == 0):
        a = (c * toolbar_width) // n
        print("[{}{}] {}%".format('■'*a, ' '*(toolbar_width - a), (c * 100) // n), end='\r', flush=True)

    c += 1

print(destinations_list)

KeyboardInterrupt: 

<h2>Test Elements - Ignore</h2>

In [181]:
indices = []
for i in range(len(list(data.keys()))):
    if(data[list(data.keys())[i]]['extra'] != None):
        indices.append(i)

print(data[list(data.keys())[indices[100]]])

{'lat': 32.164948, 'lon': -106.608966, 'baro_alt': 43000, 'grd_speed': 464, 'extra': {'flight_num': 'N628TS  ', 'geom_alt': 44000, 'baro_rate': -64, 'grd_track': 306.5}}


In [182]:
print(sorted(list(js_data[0].keys())))
print("---------------------------------------")
print(len(js_data[0]['fullTrace']['trace']))
print(js_data[0]['fullTrace']['trace'][100])
print("")
print(js_data[0]['fullTrace']['trace'][400])

['addrtype', 'alt_baro', 'alt_reliable', 'alt_rounded', 'altitude', 'altitudeTime', 'bad_alt', 'bad_altTime', 'baseMarkerKey', 'baseScale', 'bgColorCache', 'category', 'clearTraceAfter', 'country', 'dataSource', 'dbinfoLoaded', 'drawLine', 'elastic_feature', 'fakeHex', 'flag_image', 'flight', 'flightTs', 'fullTrace', 'geom_diff', 'geom_diff_ts', 'gs', 'history_size', 'icao', 'icaoType', 'icaoTypeCache', 'inView', 'interesting', 'ladd', 'last', 'lastTraceTs', 'lastVisible', 'last_info_server', 'last_message_time', 'layer', 'linesDrawn', 'marker', 'markerDrawn', 'markerIcon', 'markerStyle', 'markerStyleKey', 'markerSvgKey', 'messageRate', 'messageRateOld', 'messages', 'military', 'msgs1090', 'msgs978', 'nac_p', 'nac_v', 'name', 'nav_altitude', 'noInfoTime', 'numHex', 'onGround', 'ownOp', 'pia', 'position', 'position_time', 'prev_alt', 'prev_alt_rounded', 'prev_dataSource', 'prev_rot', 'prev_speed', 'prev_time', 'proj', 'psAPIparam', 'psAPIresponse', 'psAPIresponseTS', 'rc', 'recentTrace'

In [183]:
print(datetime.strptime("1970/01/01", '%Y/%m/%d') + timedelta(seconds=1675820888))

2023-02-08 01:48:08


In [184]:
# from selenium.webdriver.chrome.service import Service
# from time import sleep

# chromedriver = "C:/webdrivers/chromedriver.exe"

# option = webdriver.ChromeOptions()
# option.binary_location = "C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe"

# s = Service(chromedriver)

# driver = webdriver.Chrome(service=s, options=option)

# #Patch Fingerprint leaks
# script = "Object.defineProperty(navigator, 'webdriver', {get: () => false})"
# driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": script})

# delta = endDate - startDate
# for i in range(delta.days + 1):
#     driver.get("https://globe.adsbexchange.com/?icao={}&lat={}&lon={}&zoom={}&showTrace={}".format(targetIcao, lat, lon, zoom, startDate + timedelta(days=i)))
#     # sleep(10)
#     # element = driver.execute_script("return selectedPlanes()")
#     # print(element)
# # driver.close()